This code is used to visualize feature maps from the sprcific layer

In [1]:
from Model import Model
from tensorboardX import SummaryWriter
from torchvision import transforms
import torch
image_path = './example_images/000186.jpg'
from PIL import Image
import cv2
import numpy as np
from torch.autograd import Variable
import torchvision.utils as vutils

In [3]:
net = Model()
net.load_state_dict(torch.load('weights/epoch204densenet.pth'))
net.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [4]:
def preprocess_image(cv2im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): Image to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (Pytorch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    # Resize image
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [9]:
class FeatureVisualization():
    def __init__(self,img_path,selected_layer):
        self.img_path=img_path
        self.selected_layer=selected_layer
        self.pretrained_model = net.features 
        
    def process_image(self):
        img=cv2.imread(self.img_path)
        img=preprocess_image(img)
        return img

    def get_feature(self):
        # input = Variable(torch.randn(1, 3, 224, 224))
        #writer = SummaryWriter()
        input=self.process_image()
        print(input.shape)
        x=input
        for index,layer in enumerate(self.pretrained_model):
            #print('index',index, layer)
            x=layer(x)
            if (index == self.selected_layer):
                img_grid = vutils.make_grid(x, normalize=True, scale_each=True, nrow=2)  # B，C, H, W
                #writer.add_image('_feature_maps', img_grid, global_step=666)
                return x

    def get_single_feature(self, index):
        features=self.get_feature()
        print('features shape',features.shape)

        feature=features[:,index,:,:]
        print(feature.shape)

        feature=feature.view(feature.shape[1],feature.shape[2])
        print(feature.shape)

        return feature

    def save_feature_to_img(self):
        #to numpy
        for i in range(1024):
            feature=self.get_single_feature(i)
            feature=feature.data.numpy()
        
            #use sigmod to [0,1]
            feature= 1.0/(1+np.exp(-1*feature))

            # to [0,255]
            feature=np.round(feature*255)
            print(feature[0])
            feature = cv2.resize(feature, (256,256))
            feature = feature.astype(np.uint8)
            #feature = cv2.applyColorMap(feature, cv2.COLORMAP_AUTUMN)

            cv2.imwrite('./feature/8/img_%d.jpg'%(i), feature)


In [11]:
if __name__=='__main__':
    # get class
    myClass=FeatureVisualization(image_path,8)
    print (myClass.pretrained_model)

    myClass.save_feature_to_img()

Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(96, 128, kernel_size=(1, 1), st

features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[147. 132. 131. 131. 129. 130. 131. 129. 131. 131. 130. 130. 133. 140.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[127. 129. 128. 128. 130. 131. 129. 129. 127. 130. 125. 127. 131. 137.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[122. 125. 123. 126. 126. 129. 128. 128. 128. 125. 129. 125. 128. 122.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[134. 132. 135. 136. 135. 135. 133. 137. 135. 136. 134. 135. 135. 134.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[137. 136. 135. 136. 137. 137. 137. 138. 139. 134. 134. 141. 134. 139.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[126. 138. 137. 134. 133. 134. 129. 127. 131. 129. 133. 135. 132. 125.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[120. 123. 121. 120. 120. 122. 123. 122. 121. 124. 125. 121. 120. 125.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[158. 140. 140. 144. 141. 145. 144. 146. 143. 147. 145. 144. 147. 157.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[145. 137. 140. 141. 137. 135. 136. 136. 139. 138. 135. 138. 140. 145.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[143. 131. 131. 134. 134. 133. 132. 134. 135. 131. 135. 137. 133. 138.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[127. 132. 132. 131. 132. 133. 131. 134. 132. 128. 129. 131. 133. 128.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[121. 123. 124. 122. 122. 122. 122. 124. 124. 123. 124. 123. 120. 126.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[130. 131. 134. 133. 133. 131. 133. 134. 134. 132. 130. 134. 132. 137.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[141. 136. 139. 137. 133. 134. 134. 136. 136. 136. 133. 136. 137. 146.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[147. 138. 140. 137. 137. 141. 140. 141. 140. 139. 140. 144. 140. 144.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[143. 139. 142. 141. 140. 145. 141. 140. 140. 143. 140. 142. 143. 146.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[126. 125. 126. 124. 126. 127. 125. 125. 123. 125. 127. 125. 122. 123.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[134. 123. 123. 124. 125. 127. 124. 120. 120. 123. 122. 121. 123. 131.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[135. 128. 129. 127. 123. 126. 126. 122. 126. 128. 125. 126. 125. 129.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[140. 129. 131. 131. 130. 126. 129. 126. 130. 127. 128. 130. 129. 134.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


...

features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[168. 127. 134. 134. 135. 133. 134. 134. 135. 135. 132. 136. 132. 172.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[152. 135. 129. 133. 133. 134. 134. 134. 134. 134. 132. 134. 124. 153.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[172. 119. 126. 126. 125. 122. 120. 118. 117. 119. 118. 122. 102. 161.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[187. 143. 142. 150. 153. 150. 150. 149. 148. 148. 146. 147. 143. 190.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[144.  91.  97. 101. 104. 106. 106. 104. 106. 107. 105. 100.  96. 141.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[144. 118. 118. 119. 121. 122. 121. 122. 121. 121. 119. 121. 120. 146.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[123. 113. 112. 116. 119. 120. 121. 120. 120. 121. 121. 118. 111. 126.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[127. 132. 136. 134. 136. 135. 134. 133. 135. 136. 137. 141. 136. 134.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[119. 106. 123. 120. 118. 118. 120. 123. 123. 122. 119. 119.  88. 123.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[146. 119. 120. 122. 123. 123. 123. 123. 122. 122. 120. 119. 121. 144.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[180. 127. 135. 135. 137. 134. 134. 132. 132. 137. 134. 137. 121. 179.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[150. 116. 122. 125. 125. 125. 125. 126. 126. 128. 126. 118. 109. 149.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[179.  94. 115. 117. 119. 117. 119. 118. 117. 119. 115. 117.  90. 169.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[183.  99. 126. 130. 131. 131. 135. 135. 133. 134. 130. 130. 110. 193.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[210. 110. 131. 128. 129. 124. 127. 127. 127. 132. 126. 130. 106. 214.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[175. 106. 124. 124. 127. 126. 126. 126. 124. 127. 125. 127. 105. 180.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[121. 121. 116. 113. 114. 115. 116. 117. 117. 117. 115. 113. 122. 118.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[148. 114. 129. 129. 129. 126. 126. 125. 124. 124. 122. 126. 115. 151.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[167. 142. 135. 139. 139. 141. 141. 142. 142. 142. 142. 143. 155. 169.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[150. 116. 122. 124. 124. 124. 125. 123. 122. 124. 124. 125. 104. 153.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])


features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[142. 126. 129. 132. 132. 131. 129. 129. 130. 131. 132. 137. 148. 139.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[169. 124. 127. 130. 132. 131. 132. 132. 131. 134. 130. 129. 121. 168.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[190. 126. 136. 138. 140. 139. 140. 139. 141. 148. 146. 151. 141. 193.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[151. 113. 121. 125. 125. 125. 124. 124. 123. 123. 119. 117. 107. 147.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
torch.Size([1, 14, 14])
torch.Size([14, 14])
[187. 121. 132. 133. 134. 131. 133. 130. 129. 131. 128. 135. 123. 190.]
torch.Size([1, 3, 224, 224])
features shape torch.Size([1, 1024, 14, 14])
